In [1]:
import os

import numpy as np
import torch
import torch.nn as nn

from rdkit.Chem.rdmolfiles import SDMolSupplier

from featurize import AtomFeaturize, BondFeaturize, BondAngleFeaturize, BondLengthFeaturize
from utils.utils import clean_dataset, optimize_molecule
from utils.RBF import RBF
from values import VALUES

In [15]:
embed = nn.Embedding(119, 32)

In [2]:
ROOT = '..'
gdb7 = SDMolSupplier(os.path.join(ROOT, 'data/gdb7/gdb7.sdf'), removeHs=False)
gdb7 = clean_dataset(gdb7)

[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Warning: molecule is tagged as 3D, but all Z coords are zero
[18:31:28] Explicit valence for atom # 3 N, 4, is greater than permitted
[18:31:28] ERROR: Could not sanitize molecule ending on line 2301
[18:31:28] ERROR: Explicit valence for atom # 3 N, 4, is greater than permitted
[18:31:28] Explicit valence for atom # 2 N, 4, is greater than permitted
[18:31:28] ERROR: Could not sanitize molecule ending on line 2443
[18:31:28] ERROR: Explicit valence for atom # 2 N, 4, is greater than permitted
[18:31:28] Warning: molecule is tagged as 3D, but all Z co

In [4]:
ch4_features = AtomFeaturize(gdb7[0])
print(ch4_features.node_features)

{'atom_type': tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 

In [10]:
ch4_features.node_features['atom_type']

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [11]:
ch4_features.node_features['atom_type'].shape

torch.Size([5, 119])

In [18]:
embeded = embed(torch.tensor([i for i in range(0, 119)]))
print(embeded)
embeded.shape

tensor([[ 0.8512,  0.1093,  0.6513,  ..., -0.7060, -0.1030, -0.4626],
        [ 0.1475,  0.6897,  0.3501,  ..., -1.1679, -0.5875,  0.5211],
        [ 0.8071,  0.0585, -0.4455,  ...,  0.7540, -1.6052,  0.1906],
        ...,
        [-0.7358, -1.1127,  1.1234,  ..., -2.1065, -0.4382,  0.8912],
        [ 0.2446, -0.0958,  0.4989,  ...,  0.0134, -0.1220,  0.1258],
        [ 0.0122,  0.2843,  1.1732,  ...,  0.9624, -0.4275, -0.0298]],
       grad_fn=<EmbeddingBackward0>)


torch.Size([119, 32])

In [44]:
0 + torch.matmul(ch4_features.node_features['atom_type'].float(), embeded)

tensor([[ 0.9295, -0.4533, -0.0332,  0.5909, -0.2206,  0.5343,  1.4515, -0.3892,
         -1.0715, -0.5164,  0.3564,  0.4761,  0.3558,  0.0649, -1.3092,  0.0369,
          1.0989,  0.3185,  0.1554, -0.4008, -0.9119,  1.2572, -0.9987, -0.7556,
         -0.0925,  0.4704, -3.1520,  0.8275,  0.6999, -0.3922,  0.1531, -0.0992],
        [ 0.1475,  0.6897,  0.3501,  0.4651, -0.5555,  0.5515,  0.7427,  2.1498,
         -0.7010, -0.0362,  0.0694,  0.5298,  0.6058, -1.1838, -0.6401,  0.5550,
          0.6043, -0.1411,  0.3667,  0.5755,  0.5701,  0.3071,  0.4429,  0.3248,
         -0.6561,  0.0503,  1.1622, -1.9149, -0.3638, -1.1679, -0.5875,  0.5211],
        [ 0.1475,  0.6897,  0.3501,  0.4651, -0.5555,  0.5515,  0.7427,  2.1498,
         -0.7010, -0.0362,  0.0694,  0.5298,  0.6058, -1.1838, -0.6401,  0.5550,
          0.6043, -0.1411,  0.3667,  0.5755,  0.5701,  0.3071,  0.4429,  0.3248,
         -0.6561,  0.0503,  1.1622, -1.9149, -0.3638, -1.1679, -0.5875,  0.5211],
        [ 0.1475,  0.6897

In [7]:
embeded.shape

119

# Atom Embedding

In [3]:
atom_feat = AtomFeaturize(gdb7[0])
print(atom_feat.node_features)

{'atom_type': tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 

In [17]:
feat_names = list(atom_feat.node_features.keys())
embed_dim = 32

embed_list = []
for name in feat_names:
    feat_size = VALUES.get_atom_feat_size(name)
    embed = nn.Embedding(feat_size, embed_dim)
    
    embed_list.append(embed)

atom_features = atom_feat.node_features

out_embed = 0
for i, name in enumerate(feat_names):
    embed = embed_list[i](torch.tensor(VALUES.atom_feat_values[name]))
    out_embed += atom_features[name] @ embed

out_embed

tensor([[-3.7734, -0.4979,  2.7454,  0.0754,  4.1213, -3.4844, -1.2892,  2.3442,
         -1.2103,  3.0465,  3.5743, -1.5518, -1.2394,  4.5909, -2.4310,  2.0217,
          0.3393,  2.7135,  3.4966, -0.3716, -2.4158,  1.6729,  1.9663,  4.2091,
          0.4701,  1.8438,  0.2964, -3.3192,  0.9709, -0.7077, -2.3280, -4.7046],
        [-2.3062, -2.9484,  1.2384, -1.0696,  4.5437, -3.6445,  0.3220, -0.8601,
         -0.7279,  0.6059,  1.0771,  0.7009, -0.2738,  0.2472, -2.7937,  2.8504,
          1.9328,  2.0663, -0.1539, -1.5822, -1.5773, -1.3172, -2.3575,  1.0359,
          0.2615,  0.4690,  1.9803, -1.1014,  1.2767, -0.5092, -1.9458, -1.8011],
        [-3.4678, -1.4608,  2.4784, -1.2468,  3.0075, -1.7987, -1.4889,  1.5869,
         -0.9417,  1.6842,  2.4216, -3.3771,  0.2496, -1.5411, -1.4575,  2.7349,
          4.1229,  2.0687,  1.5249,  0.2315,  0.3553,  0.6459, -1.6636, -0.1362,
         -0.2357,  2.1331,  0.6824,  0.5979,  0.6378, -1.4969, -1.4596, -2.4639],
        [-2.3062, -2.9484

In [5]:
out_embed.shape

torch.Size([5, 32])

# Bond Embedding

In [18]:
bond_feat = BondFeaturize(gdb7[0])
print(bond_feat.edge_features)

{'bond_dir': tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]]), 'bond_type': tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]]), 'is_in_ring': tensor([[1., 0.],
        [1., 0.]])}


In [19]:
feat_names = list(bond_feat.edge_features.keys())
embed_dim = 32

embed_list = []
for name in feat_names:
    feat_size = VALUES.get_bond_feat_size(name)
    embed = nn.Embedding(feat_size, embed_dim)
    
    embed_list.append(embed)

bond_features = bond_feat.edge_features

out_embed = 0
for i, name in enumerate(feat_names):
    embed = embed_list[i](torch.tensor(VALUES.bond_feat_values[name]))
    out_embed += bond_features[name] @ embed

out_embed

tensor([[ 1.6487, -1.3865,  1.4285, -3.5062, -0.2686, -0.7961, -2.1418, -0.7636,
         -0.0654,  3.6038, -3.5667, -2.7945, -0.8691, -0.7967, -0.5253,  0.6004,
          0.9328, -1.0975, -0.5148, -2.8193,  1.5444, -0.8826,  0.5603, -0.1879,
         -0.6011, -1.8203,  0.5593,  0.5405,  3.1593, -2.5517,  0.2810,  1.0995],
        [ 1.6487, -1.3865,  1.4285, -3.5062, -0.2686, -0.7961, -2.1418, -0.7636,
         -0.0654,  3.6038, -3.5667, -2.7945, -0.8691, -0.7967, -0.5253,  0.6004,
          0.9328, -1.0975, -0.5148, -2.8193,  1.5444, -0.8826,  0.5603, -0.1879,
         -0.6011, -1.8203,  0.5593,  0.5405,  3.1593, -2.5517,  0.2810,  1.0995]],
       grad_fn=<AddBackward0>)

In [20]:
out_embed.shape

torch.Size([2, 32])

# Bond-length Embedding

In [4]:
bond_len = BondLengthFeaturize(gdb7[0])
bond_len.features

2


{'length': tensor([1.1175, 1.2431])}

In [5]:
embed_dim = 32
centers = list(np.arange(0, 2, 0.1))
gamma = 10

rbf = RBF(centers, gamma)
linear = nn.Linear(len(centers), embed_dim)

x = bond_len.features['length']
x = rbf(x)
x = linear(x)

x

tensor([[ 0.0262,  0.3756, -0.2643,  0.5695, -0.0371,  0.1408,  0.4998, -0.3599,
          0.0956,  0.0365, -0.2095, -0.2391, -0.0130, -0.0231, -0.4702, -0.3381,
          0.4346, -0.1333,  0.3699, -0.3363,  0.2828, -0.1812, -0.1767,  0.0307,
          0.5350, -0.1337,  0.3038,  0.2463,  0.5188, -0.1700,  0.3071,  0.3694],
        [ 0.1063,  0.3771, -0.1573,  0.3654,  0.0043,  0.1084,  0.4275, -0.4847,
          0.0736,  0.1712, -0.2232, -0.1638, -0.1821,  0.1634, -0.2335, -0.1062,
          0.4469, -0.2488,  0.1719, -0.3781,  0.3959, -0.1805, -0.1205, -0.0436,
          0.5315, -0.3370,  0.4116,  0.2761,  0.4909, -0.3443,  0.1695,  0.3719]],
       grad_fn=<AddmmBackward0>)

# Bond-angle Embedding

In [3]:
bond_angle = BondAngleFeaturize(gdb7[0])
bond_angle.features

[(1, 0, 3), (3, 0, 1)]


{'angle': tensor([0.9658, 0.9658])}

In [4]:
embed_dim = 32
centers = list(np.arange(0, torch.pi, 0.1))
gamma = 10

rbf = RBF(centers, gamma)
linear = nn.Linear(len(centers), embed_dim)

x = bond_angle.features['angle']
x = rbf(x)
x = linear(x)

x

tensor([[-0.0972, -0.0533, -0.2935, -0.3595, -0.0496,  0.0097, -0.1704,  0.3175,
         -0.1420, -0.2937,  0.2104, -0.2124, -0.1654,  0.0876,  0.3511,  0.1647,
         -0.0071, -0.0420, -0.5059,  0.0914, -0.4356,  0.4440,  0.0188, -0.1242,
          0.2968, -0.4563, -0.2832,  0.2401,  0.1085, -0.1465,  0.0449,  0.1561],
        [-0.0972, -0.0533, -0.2935, -0.3595, -0.0496,  0.0097, -0.1704,  0.3175,
         -0.1420, -0.2937,  0.2104, -0.2124, -0.1654,  0.0876,  0.3511,  0.1647,
         -0.0071, -0.0420, -0.5059,  0.0914, -0.4356,  0.4440,  0.0188, -0.1242,
          0.2968, -0.4563, -0.2832,  0.2401,  0.1085, -0.1465,  0.0449,  0.1561]],
       grad_fn=<AddmmBackward0>)